<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto Part_2</font></h1>

<h2 align=center><font size = 4>Assignment Week 3</font></h2>

In [1]:
#Import Libs
!conda install -c conda-forge geopy --yes 
import json
import folium
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values (OpenStreetMaps)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#import csv file created in Part_1
df = pd.read_csv('toronto_geocoded.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


"Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you."

In [3]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
# Having geograpical coordinate lets create map of Toronto using latitude and longitude values
neighborhoods = df
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# Adding markers to map
for lat, lng, borough, neighbourhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto


In [5]:
#FourSquare loading
CLIENT_ID = 'PGOXNIZB4V20LRDNVE333WDXHHTCFE0THMGDV55XKLPQSE04' # your Foursquare ID
CLIENT_SECRET = 'FLOOZKBCWHFDRKYSMHHLETU1DKNQZLTAEM31IFSBIIR1LXXE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30


In [6]:
df_new = pd.DataFrame([[CLIENT_ID,CLIENT_SECRET,VERSION]],columns=['CLIENT_ID','CLIENT_SECRET','VERSION'])
df_new.to_csv('foursquare_data.csv')
df_data = pd.read_csv('foursquare_data.csv')
df_data

,Unnamed: 0,CLIENT_ID,CLIENT_SECRET,VERSION
0,0,PGOXNIZB4V20LRDNVE333WDXHHTCFE0THMGDV55XKLPQSE04,FLOOZKBCWHFDRKYSMHHLETU1DKNQZLTAEM31IFSBIIR1LXXE,20180604


In [7]:
#GET FOURSQUARE KEY DATA 
data = pd.read_csv('foursquare_data.csv')
data['CLIENT_ID']= data['CLIENT_ID'].astype(str)
data['CLIENT_SECRET']= data['CLIENT_SECRET'].astype(str)
data['VERSION']= data['VERSION'].astype(str)
CLIENT_ID = data.iloc[0]['CLIENT_ID']
CLIENT_SECRET = data.iloc[0]['CLIENT_SECRET']
VERSION = data.iloc[0]['VERSION']

def getBuiltUrl(neigh_lat,neigh_long,radius=1400):
    # type your answer here
    LIMIT=100
    #radius=1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url
    #url

In [8]:

df.shape

(103, 5)

In [9]:
# get one test Neighbourhood
neigh_post, neigh_borough, neigh_name, neigh_lat, neigh_long = df.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))
results = requests.get(getBuiltUrl(neigh_lat,neigh_long)).json()
results

Latitude and longitude values of Rouge, Malvern are 43.8066863, -79.19435340000003.


{'meta': {'code': 200, 'requestId': '5dc1f82c5d891b002c87c9c4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 30,
  'suggestedBounds': {'ne': {'lat': 43.81928631260001,
    'lng': -79.17692670604515},
   'sw': {'lat': 43.79408628739999, 'lng': -79.2117800939549}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': 

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()


,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374
2,LCBO,Liquor Store,43.796671,-79.204586
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777


In [16]:
#venues
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = getBuiltUrl(lat,lng,radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#neighborhoods = neighborhoods.drop(neighborhoods.columns[0], axis=1)
neighborhoods['Neighbourhood'] = neighborhoods['Neighbourhood']+' '+neighborhoods['PostalCode']
print(neighborhoods['Neighbourhood'].shape)
neighborhoods.head()

(103,)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern M1B M1B",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union M1C M1C",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill M1E M1E",43.763573,-79.188711
3,M1G,Scarborough,Woburn M1G M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae M1H M1H,43.773136,-79.239476


In [22]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'], 
                                   latitudes=neighborhoods['Latitude'], 
                                   longitudes=neighborhoods['Longitude'], 
                                   radius=4000)

Rouge, Malvern M1B M1B
Highland Creek, Rouge Hill, Port Union M1C M1C
Guildwood, Morningside, West Hill M1E M1E
Woburn M1G M1G
Cedarbrae M1H M1H
Scarborough Village M1J M1J
East Birchmount Park, Ionview, Kennedy Park M1K M1K
Clairlea, Golden Mile, Oakridge M1L M1L
Cliffcrest, Cliffside, Scarborough Village West M1M M1M
Birch Cliff, Cliffside West M1N M1N
Dorset Park, Scarborough Town Centre, Wexford Heights M1P M1P
Maryvale, Wexford M1R M1R
Agincourt M1S M1S
Clarks Corners, Sullivan, Tam O'Shanter M1T M1T
Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V M1V
L'Amoreaux West M1W M1W
Upper Rouge M1X M1X
Hillcrest Village M2H M2H
Fairview, Henry Farm, Oriole M2J M2J
Bayview Village M2K M2K
Silver Hills, York Mills M2L M2L
Newtonbrook, Willowdale M2M M2M
Willowdale South M2N M2N
York Mills West M2P M2P
Willowdale West M2R M2R
Parkwoods M3A M3A
Don Mills North M3B M3B
Flemingdon Park, Don Mills South M3C M3C
Bathurst Manor, Downsview North, Wilson Heights M3H M3H
Northwood Park, 

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(10187, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern M1B M1B",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
1,"Rouge, Malvern M1B M1B",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
2,"Rouge, Malvern M1B M1B",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
3,"Rouge, Malvern M1B M1B",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
4,"Rouge, Malvern M1B M1B",43.806686,-79.194353,Australasia Pavillion,43.822563,-79.183286,Zoo Exhibit


In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond M5H M5H",100,100,100,100,100,100
Agincourt M1S M1S,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V M1V",100,100,100,100,100,100
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown M9V M9V",100,100,100,100,100,100
"Alderwood, Long Branch M8W M8W",100,100,100,100,100,100
"Bathurst Manor, Downsview North, Wilson Heights M3H M3H",100,100,100,100,100,100
Bayview Village M2K M2K,100,100,100,100,100,100
"Bedford Park, Lawrence Manor East M5M M5M",100,100,100,100,100,100
Berczy Park M5E M5E,100,100,100,100,100,100


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques Neighborhoods.'.format(len(toronto_venues['Neighborhood'].unique())))

There are 276 uniques categories.
There are 103 uniques Neighborhoods.


In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()
#toronto_venues['Neighborhood'].size

(10187, 276)


,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Liquor Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(103, 276)


,Neighborhood,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Liquor Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide, King, Richmond M5H M5H",0.0,0.00,0.0,0.0,

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond M5H M5H----
                venue  freq
0         Coffee Shop  0.07
1               Hotel  0.05
2  Italian Restaurant  0.05
3                Café  0.04
4         Pizza Place  0.04


----Agincourt M1S M1S----
                  venue  freq
0    Chinese Restaurant  0.09
1           Coffee Shop  0.06
2  Caribbean Restaurant  0.05
3           Supermarket  0.04
4     Indian Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V M1V----
                  venue  freq
0    Chinese Restaurant  0.14
1       Bubble Tea Shop  0.05
2           Coffee Shop  0.05
3                Bakery  0.05
4  Caribbean Restaurant  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown M9V M9V----
                  venue  freq
0           Coffee Shop  0.15
1  Fast Food Restaurant  0.10
2     Indian Restaurant  0.07
3        Sandwich Place  0.06
4                  Bank  0.05


----Alderwood, Lon

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond M5H M5H",Coffee Shop,Italian Restaurant,Hotel,Pizza Place,Café,Sandwich Place,Restaurant,Japanese Restaurant,Farmers Market,Sporting Goods Shop
1,Agincourt M1S M1S,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Supermarket,Indian Restaurant,Fast Food Restaurant,Restaurant,Clothing Store,Bakery,Bubble Tea Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Bakery,Bubble Tea Shop,Caribbean Restaurant,Vietnamese Restaurant,Dessert Shop,Supermarket,Asian Restaurant,Indian Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Bank,Pizza Place,Grocery Store,Asian Restaurant,Caribbean Restaurant,Pharmacy
4,"Alderwood, Long Branch M8W M8W",Coffee Shop,Bakery,Burger Joint,Liquor Store,Pizza Place,Café,Breakfast Spot,Grocery Store,Furniture / Home Store,Seafood Restaurant


<h2 align=center><font size = 4> K-Clustering </font></h2>

In [37]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#toronto_grouped_clustering.head()

array([0, 7, 7, 3, 5, 4, 8, 4, 0, 2], dtype=int32)

In [38]:
toronto_merged = neighborhoods#.merge(toronto_grouped['Neighborhood'], how='right', on='Neighborhood')
toronto_merged = toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'})

In [39]:

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern M1B M1B",43.806686,-79.194353,0,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pharmacy,Pizza Place,Breakfast Spot,Burger Joint,Chinese Restaurant,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union M1C M1C",43.784535,-79.160497,7,Coffee Shop,Pizza Place,Park,Fast Food Restaurant,Sandwich Place,Pharmacy,Breakfast Spot,Trail,Fried Chicken Joint,Beer Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill M1E M1E",43.763573,-79.188711,7,Coffee Shop,Pizza Place,Sandwich Place,Fast Food Restaurant,Burger Joint,Pharmacy,Fried Chicken Joint,Beer Store,Park,Indian Restaurant
3,M1G,Scarborough,Woburn M1G M1G,43.770992,-79.216917,3,Coffee Shop,Indian Restaurant,Clothing Store,Pizza Place,Fast Food Restaurant,Gym,Caribbean Restaurant,Pharmacy,Park,Bank
4,M1H,Scarborough,Cedarbrae M1H M1H,43.773136,-79.239476,5,Indian Restaurant,Coffee Shop,Park,Caribbean Restaurant,Train Station,Steakhouse,Fast Food Restaurant,Breakfast Spot,Sandwich Place,Asian Restaurant


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2 align=center><font size = 4> Analysing Obtained Clusters </font></h2>

In [45]:
clust_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_0.head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pharmacy,Pizza Place,Breakfast Spot,Burger Joint,Chinese Restaurant,Discount Store
8,Scarborough,0,Coffee Shop,Fast Food Restaurant,Park,Chinese Restaurant,Bank,Pizza Place,Pharmacy,Sandwich Place,Gym,Beer Store
14,Scarborough,0,Chinese Restaurant,Coffee Shop,Bakery,Bubble Tea Shop,Caribbean Restaurant,Vietnamese Restaurant,Dessert Shop,Supermarket,Asian Restaurant,Indian Restaurant
15,Scarborough,0,Chinese Restaurant,Bakery,Caribbean Restaurant,Bubble Tea Shop,Japanese Restaurant,Noodle House,Asian Restaurant,Sushi Restaurant,Korean Restaurant,Coffee Shop
19,North York,0,Korean Restaurant,Coffee Shop,Bakery,Middle Eastern Restaurant,Café,Sushi Restaurant,Thai Restaurant,Bubble Tea Shop,Supermarket,Furniture / Home Store


In [46]:
clust_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_1 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Scarborough,1,Zoo Exhibit,Fast Food Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Grocery Store,Pharmacy,Skating Rink,Burger Joint,Golf Course
21,North York,1,Korean Restaurant,Coffee Shop,Middle Eastern Restaurant,Café,Supermarket,Sushi Restaurant,Bubble Tea Shop,Grocery Store,Bagel Shop,Liquor Store
30,North York,1,Clothing Store,Coffee Shop,Restaurant,Grocery Store,Cosmetics Shop,Furniture / Home Store,Liquor Store,Dessert Shop,Deli / Bodega,Athletics & Sports
35,East York,1,Gastropub,Café,Bakery,Brewery,Middle Eastern Restaurant,Gym / Fitness Center,Coffee Shop,Indian Restaurant,Turkish Restaurant,Thai Restaurant
46,Central Toronto,1,Coffee Shop,Italian Restaurant,Bakery,Café,Park,Gym,Sushi Restaurant,Breakfast Spot,Gastropub,Japanese Restaurant
52,Downtown Toronto,1,Café,Coffee Shop,Japanese Restaurant,Park,Pizza Place,Restaurant,Italian Restaurant,Hotel,Museum,Thai Restaurant
58,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Hotel,Pizza Place,Café,Sandwich Place,Restaurant,Japanese Restaurant,Farmers Market,Sporting Goods Shop
68,Downtown Toronto,1,Park,Hotel,Pizza Place,Bakery,Yoga Studio,Coffee Shop,Beer Bar,Italian Restaurant,Bar,Theater
86,Mississauga,1,Hotel,Coffee Shop,Sandwich Place,Pharmacy,Japanese Restaurant,Middle Eastern Restaurant,Grocery Store,Vietnamese Restaurant,Asian Restaurant,Burger Joint


In [48]:
clust_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_2.head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,2,Park,Coffee Shop,Breakfast Spot,Beach,Pub,Bar,Bank,Caribbean Restaurant,Bakery,Tea Room
10,Scarborough,2,Coffee Shop,Middle Eastern Restaurant,Supermarket,Chinese Restaurant,Clothing Store,Pharmacy,Indian Restaurant,Pizza Place,Gym,Burger Joint
17,North York,2,Chinese Restaurant,Bakery,Japanese Restaurant,Coffee Shop,Caribbean Restaurant,Breakfast Spot,Asian Restaurant,Pharmacy,Sushi Restaurant,Sporting Goods Shop
18,North York,2,Chinese Restaurant,Bakery,Coffee Shop,Caribbean Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Supermarket,Steakhouse,Tea Room,Seafood Restaurant
23,North York,2,Coffee Shop,Grocery Store,Sushi Restaurant,Café,Bubble Tea Shop,Bakery,Bagel Shop,Pub,Burger Joint,Park


In [ ]:
clust_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_3.head()

In [ ]:
clust_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_4.head()

In [ ]:
clust_5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_5.head()

In [ ]:
clust_6 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_6.head()

In [29]:
clust_7 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_7.head()

NameError: name 'toronto_merged' is not defined

In [ ]:
clust_8 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_8.head()

In [ ]:
clust_9 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
clust_9.head()